MINE paper: https://arxiv.org/pdf/1801.04062.pdf

In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import sys
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd

import pycuda.driver as cuda

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
use_cuda = torch.cuda.is_available()
print("Setting Info")
print("=========")
print("- use_cuda: ", use_cuda)
print("- Path: ", os.getcwd())
print("- PyTorch", torch.__version__)
print("- Python: ", sys.version)

Setting Info
- use_cuda:  True
- Path:  /home/uchiumi/JNNS2019/mnist_pytorch
- PyTorch 1.0.1.post2
- Python:  3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609]


In [11]:
n_epochs = 10
batch_size_train = 128
batch_size_test = 1000
learning_rate = 0.01
momentum = 0.5
log_interval = 100

random_seed = 1
torch.backends.cudnn.enabled = False

In [12]:
# train
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', 
                             train=True, 
                             download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

# test
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', 
                             train=False, 
                             download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

In [13]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)

In [14]:
example_data.shape

torch.Size([1000, 1, 28, 28])

## Building the Network

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        layer_values={}
        layer_values["input_image"]=x
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        layer_values["conv1_output"]=x
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        layer_values["conv2_output"]=x
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        layer_values["fc1_output"]=x
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        layer_values["fc2_output"]=x
        x = F.log_softmax(x)
        layer_values["output_softmax"]=x
        
        return layer_values

## Training the Model

In [16]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)

In [17]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(1, n_epochs+1)]

In [18]:
def model_train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)["output_softmax"]
        train_loss = F.nll_loss(output, target)
        train_loss.backward()
        optimizer.step()
        
        if batch_idx % log_interval == 0 and batch_idx != 0:
            print('epoch: {} [{}/{} ]\t train loss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), train_loss.item()))
            
            train_losses.append(train_loss.item())
            train_counter.append((batch_idx * batch_size_train) + ((epoch - 1)*len(train_loader.dataset)))
            
            torch.save(network.state_dict(), './results/model.pth')
            torch.save(optimizer.state_dict(), './results/optimizer.pth')

In [19]:
def model_test():
    network.eval()
    test_loss = 0
    nb_correct = 0
    
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = network(data)["output_softmax"]
            test_loss += F.nll_loss(output, target, size_average=False).item()
            test_batch_loss = F.nll_loss(output, target, size_average=False).item() / batch_size_test
            # test_loss /= len(test_loader.dataset)
            # test_losses.append(test_loss)
            pred = output.data.max(1, keepdim=True)[1]
            nb_correct += pred.eq(target.data.view_as(pred)).sum()
            print('Test set: Avg. loss: {:.4f}, Accuracy: {}/{}'.format(test_batch_loss, nb_correct, batch_size_test * (batch_idx + 1)))
    
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

In [20]:
for epoch in range(1, n_epochs + 1):
    model_train(epoch)
    model_test()

/home/uchiumi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 1 [12800/60000 ]	 train loss: 1.787463
epoch: 1 [25600/60000 ]	 train loss: 1.045102
epoch: 1 [38400/60000 ]	 train loss: 0.814315
epoch: 1 [51200/60000 ]	 train loss: 0.726596


/home/uchiumi/.local/lib/python3.5/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test set: Avg. loss: 0.2915, Accuracy: 924/1000
Test set: Avg. loss: 0.2736, Accuracy: 1855/2000
Test set: Avg. loss: 0.2432, Accuracy: 2786/3000
Test set: Avg. loss: 0.3212, Accuracy: 3689/4000
Test set: Avg. loss: 0.2949, Accuracy: 4601/5000
Test set: Avg. loss: 0.3000, Accuracy: 5516/6000
Test set: Avg. loss: 0.2891, Accuracy: 6435/7000
Test set: Avg. loss: 0.2770, Accuracy: 7356/8000
Test set: Avg. loss: 0.2904, Accuracy: 8271/9000
Test set: Avg. loss: 0.3074, Accuracy: 9188/10000
epoch: 2 [12800/60000 ]	 train loss: 0.451895
epoch: 2 [25600/60000 ]	 train loss: 0.551233
epoch: 2 [38400/60000 ]	 train loss: 0.546169
epoch: 2 [51200/60000 ]	 train loss: 0.490027
Test set: Avg. loss: 0.1550, Accuracy: 955/1000
Test set: Avg. loss: 0.1660, Accuracy: 1904/2000
Test set: Avg. loss: 0.1504, Accuracy: 2856/3000
Test set: Avg. loss: 0.1833, Accuracy: 3802/4000
Test set: Avg. loss: 0.1898, Accuracy: 4752/5000
Test set: Avg. loss: 0.1764, Accuracy: 5700/6000
Test set: Avg. loss: 0.1867, Accu

KeyboardInterrupt: 

## Evaluating the Model's Performance

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.plot(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')
plt.show()

## Get nodes values

https://www.aiworkbox.com/lessons/examine-mnist-dataset-from-pytorch-torchvision

In [26]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [27]:
X_train_0 = np.asarray(mnist_trainset[0][0]) # image
y_train_0 = mnist_trainset[0][1] # label

In [28]:
# train
train_loader_for_MI = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

# test
test_loader_for_MI = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./data', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=1, shuffle=False)

In [22]:
# model reload

PATH = "/home/uchiumi/JNNS2019/mnist_pytorch/results/model__2019-0422-1237.pth"

model = Net()
model.load_state_dict(torch.load(PATH))

In [23]:
def train_data_for_MI(model):
    model.eval()
    list = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(train_loader_for_MI):
            result = model(data)
            list.append(result)
    return list

In [24]:
def test_data_for_MI(model):
    model.eval()
    list = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader_for_MI):            
            result = model(data)
            list.append(result)
    return list

In [29]:
list = train_data_for_MI(model)

/home/uchiumi/.local/lib/python3.5/site-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [30]:
len(list)

60000

In [31]:
list[0].keys()

dict_keys(['conv1_output', 'conv2_output', 'output_softmax', 'input_image', 'fc1_output', 'fc2_output'])

In [32]:
print("input_image", list[0]["input_image"].shape)
print(list[0]["conv1_output"].shape)
print(list[0]["conv2_output"].shape)
print(list[0]["fc1_output"].shape)
print(list[0]["fc2_output"].shape)
print(list[0]["output_softmax"].shape)

input_image torch.Size([1, 1, 28, 28])
torch.Size([1, 10, 12, 12])
torch.Size([1, 20, 4, 4])
torch.Size([1, 50])
torch.Size([1, 10])
torch.Size([1, 10])


In [23]:
tensordata_input_image = []
tensordata_conv1_output = []
tensordata_conv2_output = []
tensordata_fc1_output = []
tensordata_fc2_output = []
tensordata_output_softmax = []

for i in range(len(train_loader_for_MI)):
    tensordata_input_image.append(list[i]["input_image"].data.numpy().flatten())
    tensordata_conv1_output.append(list[i]["conv1_output"].data.numpy().flatten())
    tensordata_conv2_output.append(list[i]["conv2_output"].data.numpy().flatten())
    tensordata_fc1_output.append(list[i]["fc1_output"].data.numpy().flatten())
    tensordata_fc1_output.append(list[i]["fc2_output"].data.numpy().flatten())
    tensordata_output_softmax.append(list[i]["output_softmax"].data.numpy().flatten())


In [24]:
tensordata_input_image = np.array(tensordata_input_image)
tensordata_conv1_output = np.array(tensordata_conv1_output)
tensordata_conv2_output = np.array(tensordata_conv2_output)
tensordata_fc1_output = np.array(tensordata_fc1_output)
tensordata_fc2_output = np.array(tensordata_fc2_output)
tensordata_output_softmax = np.array(tensordata_output_softmax)

## MINE (試作ver.)

In [25]:
x = tensordata_conv1_output
y = tensordata_conv2_output
z = np.concatenate([x, y], axis=1)

In [26]:
print(x.shape)
print(y.shape)
print(z.shape)

(60000, 1440)
(60000, 320)
(60000, 1760)


In [27]:
def sample_batch(x, y, sample_size, batch_size=int(1e2), sample_mode='joint'):
    if sample_mode == 'joint':
        index_1 = np.random.choice(range(sample_size), size=batch_size, replace=False)
        z = np.concatenate([x, y], axis=1)
        batch = z[index_1]
    elif sample_mode == 'marginal':
        index_1 = np.random.choice(range(sample_size), size=batch_size, replace=False)
        index_2 = np.random.choice(range(sample_size), size=batch_size, replace=False)
        batch = np.concatenate([x[index_1], y[index_2]], axis=1)
        
    return batch

In [28]:
# joint & marginal sample
joint_data = sample_batch(x, y, x.shape[0], batch_size=1000, sample_mode='joint')
marginal_data = sample_batch(x, y, x.shape[0], batch_size=1000,sample_mode='marginal')

print(joint_data.shape)
print(marginal_data.shape)

(1000, 1760)
(1000, 1760)


In [35]:
class StochasticNet(nn.Module):
    def __init__(self, xdim=1, ydim=1, hidden_size=10000):
        super().__init__()
        self.xdim = xdim
        self.ydim = ydim
        
        self.fc1 = nn.Linear(self.xdim+self.ydim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 1)
        nn.init.normal_(self.fc1.weight, std=0.02)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.normal_(self.fc2.weight, std=0.02)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.normal_(self.fc3.weight, std=0.02)
        nn.init.constant_(self.fc3.bias, 0)
        nn.init.normal_(self.fc4.weight, std=0.02)
        nn.init.constant_(self.fc4.bias, 0)
        nn.init.normal_(self.fc5.weight, std=0.02)
        nn.init.constant_(self.fc5.bias, 0)
        
    def forward(self, input_):
        output = F.relu(self.fc1(input_))
        output = F.dropout(output, p=0.2, training=True)
        output = F.relu(self.fc2(output))
        output = F.dropout(output, p=0.2, training=True)
        output = F.relu(self.fc3(output))
        output = F.dropout(output, p=0.2, training=True)
        output = F.relu(self.fc4(output))
        output = F.dropout(output, p=0.2, training=True)
        output = self.fc5(output)
        return output

In [30]:
def calc_MI_LowerBound(joint, marginal, net):
    t = net(joint)
    et = torch.exp(net(marginal))
    mi_lb = torch.mean(t) - torch.log(torch.mean(et)) # Lower bound for MI
    return mi_lb, t, et

In [31]:
def update(joint_batch, marginal_batch, net, optimizer, ma_et, ma_rate=0.01):
    # batch is a tuple of (joint, marginal)
    joint_batch = torch.autograd.Variable(torch.FloatTensor(joint_batch)).cuda()
    marginal_batch = torch.autograd.Variable(torch.FloatTensor(marginal_batch)).cuda()
    
    mi_lb , t, et = calc_MI_LowerBound(joint_batch, marginal_batch, net)
    
    # unbiasing use moving average
    ma_et = (1 - ma_rate) * ma_et + ma_rate * torch.mean(et)
    loss = -1 * (torch.mean(t) - (1 / ma_et.mean()).detach() * torch.mean(et)) # original loss function
    # use biased estimator
    # loss = - mi_lb
    
    optimizer.zero_grad()
    autograd.backward(loss)
    optimizer.step()
    
    return mi_lb, ma_et, loss

In [37]:
def train(x, y, net, optimizer, batch_size=1000, nb_epoch=int(5e+4), log_freq=int(5e+2)):

    mi_lower_bounds = []
    losses = []
    # moving average of exp(T)
    ma_et = 1.
    
    for i in range(nb_epoch):
        if x.shape[0] != y.shape[0]: 
            print("shape error.")
            break
            
        sample_size = x.shape[0]
        joint_batch = sample_batch(x, y, sample_size, batch_size=batch_size, sample_mode='joint')
        marginal_batch = sample_batch(x, y, sample_size, batch_size=batch_size, sample_mode='marginal')
        
        mi_lb, ma_et, loss = update(joint_batch, marginal_batch, net, optimizer, ma_et)
        mi_lower_bounds.append(mi_lb.detach().cpu().numpy())
        losses.append(loss)
        
        if (i + 1) % (log_freq) == 0:
            print('epoch: {:>6} \t MI lower bounds: {:2.4f} \t Loss of MINE: {:2.4f}'.format(i+1, mi_lower_bounds[-1], losses[-1]))
            
    return mi_lower_bounds, losses

In [33]:
# moving average
def ma(array, window_size=100):
    return [np.mean(array[i : i + window_size]) for i in range(0, len(array) - window_size)]

In [ ]:
net = StochasticNet(xdim=x.shape[1], ydim=y.shape[1]).cuda()
optimizer = optim.Adam(net.parameters(), lr=1e-4)

mi_lower_bounds, losses = train(x, y, net, optimizer)

epoch:    500 	 MI lower bounds: 6.9652 	 Loss of MINE: -2.8194
epoch:   1000 	 MI lower bounds: 4.6877 	 Loss of MINE: -2.6997
epoch:   1500 	 MI lower bounds: 1.4051 	 Loss of MINE: 0.5222
epoch:   2000 	 MI lower bounds: 6.8507 	 Loss of MINE: -2.7026
epoch:   2500 	 MI lower bounds: 8.3592 	 Loss of MINE: -4.3389
epoch:   3000 	 MI lower bounds: 3.9325 	 Loss of MINE: -0.5678
epoch:   3500 	 MI lower bounds: 6.8348 	 Loss of MINE: -2.6565
epoch:   4000 	 MI lower bounds: 7.1630 	 Loss of MINE: -2.4320
epoch:   4500 	 MI lower bounds: 6.1829 	 Loss of MINE: -1.2273
epoch:   5000 	 MI lower bounds: 2.9296 	 Loss of MINE: 2.3008
epoch:   5500 	 MI lower bounds: 6.6761 	 Loss of MINE: -0.8939
epoch:   6000 	 MI lower bounds: 8.8243 	 Loss of MINE: -2.2747
epoch:   6500 	 MI lower bounds: 6.1483 	 Loss of MINE: -0.3390
epoch:   7000 	 MI lower bounds: 7.3269 	 Loss of MINE: -1.0052
epoch:   7500 	 MI lower bounds: 6.8829 	 Loss of MINE: -0.4273
epoch:   8000 	 MI lower bounds: 4.4865 	 

In [ ]:
# MI lower boundsのプロット

mi_lower_bound_ma = ma(mi_lower_bounds, window_size=200)

plt.title("Moving Average of MI(X, Y) \n  (window_size=200)")
plt.xlabel("epoch")
plt.ylabel("MA of MI(X, Y)")
plt.plot(range(len(mi_lower_bound_ma)), mi_lower_bound_ma)
plt.show()

print("Final value of MI: ", mi_lower_bound_ma[-1])

In [ ]:
# MI lower boundsのプロット

mi_lower_bound_ma = ma(mi_lower_bounds, window_size=200)

plt.title("Moving Average of MI(X, Y) \n  (window_size=200)")
plt.xlabel("epoch")
plt.ylabel("MA of MI(X, Y)")
plt.plot(range(len(mi_lower_bound_ma)), mi_lower_bound_ma)
plt.show()

print("Final value of MI: ", mi_lower_bound_ma[-1])

In [ ]:
# Lossのプロット

plt.title("Loss of MINE \n  (window_size=200)")
plt.xlabel("epoch")
plt.ylabel("MA of MI(X, Y)")
plt.plot(range(len(losses)), losses)
plt.show()

print("Final value of Loss: ", losses[-1])